In [1]:
import os
import re
import calendar
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Output directory
output_dir = "./demo"
os.makedirs(output_dir, exist_ok=True)

# Function to determine financial year
def get_financial_year(year: int, month: str) -> str:
    first_quarter = ['January', 'February', 'March']
    start_year = year - 1 if month in first_quarter else year
    end_year = str(start_year + 1)[-2:]
    return f"{start_year}-{end_year}"

# Loop through April 2020 to March 2025
for y in range(2020, 2026):
    for m in range(1, 13):
        # Skip data before April 2020
        if y == 2020 and m < 4:
            continue
        if y == 2025 and m > 3:
            break

        # Process year and month
        year = y
        month_num = m
        month_name = calendar.month_name[m]
        financial_year = get_financial_year(year, month_name)

        print(f"Processing {year}-{month_num:02d} ... Financial year: {financial_year}")

        # Step 1: Visit the webpage and locate the corresponding CSV link
        url = f"https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/ae-attendances-and-emergency-admissions-{financial_year}/"
        try:
            response = requests.get(url)
            response.raise_for_status()
        except Exception as e:
            print(f"Failed to access {url}: {e}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")

        # Find matching h3 headers
        h3_tags = soup.find_all("h3")
        pattern = fr"{month_name}\s+\d{{4}}"   # e.g., "April 2020"

        csv_url = None
        for h3_tag in h3_tags:
            header_text = h3_tag.text.strip()
            if re.match(pattern, header_text):
                csv_link = h3_tag.find_next_sibling("p").find("a", href=lambda href: href and href.endswith(".csv"))
                if csv_link:
                    csv_url = csv_link["href"]
                    break

        if not csv_url:
            print(f"No CSV link found for {year}-{month_num:02d}")
            continue

        print(f"Found CSV URL: {csv_url}")

        # Step 2: Download the CSV and process it
        try:
            df = pd.read_csv(csv_url)
        except Exception as e:
            print(f"Failed to read CSV {csv_url}: {e}")
            continue

        # Clean: remove 'Total' rows and drop empty columns
        df_filtered = df[~df['Period'].str.contains('Total', case=False, na=False)]
        df_filtered = df_filtered.dropna(axis=1, how='all')

        # 🆕 Add year_month column
        year_month_value = f"{year}-{month_num:02d}-15"
        df_filtered['year_month'] = year_month_value

        # Save to CSV
        output_path = os.path.join(output_dir, f"{year}-{month_num:02d}.csv")
        df_filtered.to_csv(output_path, index=False)
        print(f"Saved to {output_path}")

print("All done!")

Processing 2020-04 ... Financial year: 2020-21
✅ Found CSV URL: https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/05/April-2020-revised-260421-abc123.csv
✅ Saved to ./demo\2020-04.csv
Processing 2020-05 ... Financial year: 2020-21
✅ Found CSV URL: https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/05/May-2020-revised-260421-de345.csv
✅ Saved to ./demo\2020-05.csv
Processing 2020-06 ... Financial year: 2020-21
✅ Found CSV URL: https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/05/June-2020-revised-260421-df453.csv
✅ Saved to ./demo\2020-06.csv
Processing 2020-07 ... Financial year: 2020-21
✅ Found CSV URL: https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/05/July-2020-revised-270421-cd305.csv
✅ Saved to ./demo\2020-07.csv
Processing 2020-08 ... Financial year: 2020-21
✅ Found CSV URL: https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/05/August-2020-revised-270421-gh920.csv
✅ Saved to ./demo